# Salvando um imagem do GEE em GeoTIFF (separado por bandas)

Neste notebook, com os dados do notebook sobre a aplicação de indices espectrais em uma imagem extraída do GEE, iremos salvá-la em GeoTIFF usando link de download fornecido pela plataforma.

Primeiramente, vamos importar a biblioteca e inicializá-la do GEE:

In [1]:
import ee # importação
import requests
ee.Initialize() # inicialização

Agora, vamos definir a geometria e as datas (baseada na Latitude e Longitude) da nossa área de estudo e consultá-la no GEE:

In [2]:
# Notem que foi criada uma coordenada (Latitude e Longitude) através de uma string, posteriormente repartida pelas virgulas
# Essa abordagem é importante para quando utilizarmos a linha da comando
coordenadas = "-48.53801472648439,-22.503806214013736,-48.270222978437516,-22.7281869567509"

# Aqui, usamos uma ferramenta do Python chamada de unpacking
x1,y1,x2,y2 = coordenadas.split(",")

# Criamos a geometria com base nas coordenadas 'quebradas' acima
geometria = geometry = ee.Geometry.Polygon(
        [[[float(x1),float(y2)],
          [float(x2),float(y2)],
          [float(x2),float(y1)],
          [float(x1),float(y1)],
          [float(x1),float(y2)]]])

# String de datas
datas = "2014-10-13,2014-10-14"

# Divisão das duas datas pela vírgula, novamente usando a técnica de unpacking
inicio,fim = datas.split(",")

# Consultando a coleção com base na área de estudo e datas selecionadas
colecao = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(geometria).filterDate(inicio,fim).filterMetadata('CLOUD_COVER','less_than', 30)

# Mostrar o total de imagens encontradas
print("Total de imagens encontradas: "+str(colecao.size().getInfo()))

Total de imagens encontradas: 1


Criamos então uma função chamada 'ndvi' que será aplicada à coleção:

In [3]:
# Definimos aqui as bandas que serão utilizadas (colocamos em uma variável para facilitar o entendimento)
# As bandas e suas nomenclaturas devem ser consultadas no manual do sensor na documentaçõa do GEE
red = 'B4'
nir = 'B5'

# Função para aplicar à imagem vinda da coleção a equação do NDVI
# O GEE apresenta essa função denominada .expression que permite que se faça operações aritiméticas com as bandas
# O retorna será a própria imagem acrescida da banda 'ndvi', permitindo que façamos uso dela posteriormente
def ndvi(imagem):
    ndvi = imagem.expression('(nir - red) / (nir + red)',{'nir':imagem.select(nir),'red':imagem.select(red)}).rename('ndvi')
    return imagem.addBands(ndvi)

In [4]:
# aplicar a função 'ndvi' em todas as imagens (irá adicionar uma nova banda em cada uma delas):
colecao = colecao.map(ndvi)

# extraindo a iamgem mediana da coleção
imagem = colecao.median()

# Podemos listar as bandas disponíveis na imagem, verificando, inclusive, a banda 'ndvi' recém adicionada.
print(imagem.bandNames().getInfo())

# Cortando a imagem
# scale = escala do sensor. No caso do Landsat-8/OLI são 30 metros
imagem_corte = imagem.clipToBoundsAndScale(geometry=geometria,scale=30)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa', 'ndvi']


Podemos então salvar a imagem cortada em um GeoTIFF usando a função de download do GEE:

In [7]:
# Salvar o zip do tiff rgb
# Notem que no zip, você encontrará cada uma das bandas separadas por arquivo geotiff
# O Google Earth Engine API adiciona um limite máximo para download (não pode ser todas as bandas). Portanto, selecione as bandas desejadas para não receber um erro
open('tiffs/12-geotiff.zip', 'wb').write(requests.get(imagem_corte.select(['B4', 'B3', 'B2','ndvi']).getDownloadUrl(), allow_redirects=True).content)

6717837

### Verifiquem a pasta 'tiffs' para o resultado!